In [2]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)
img = cv2.imread("sources/swap.jpg", cv2.IMREAD_GRAYSCALE)

MIN_MATCH_COUNT = 10
sift = cv2.SIFT_create()
kp_t, des_t = sift.detectAndCompute(img, None)
if des_t is not None and des_t.dtype != np.float32:
  des_t = des_t.astype(np.float32)

FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)
flann = cv2.FlannBasedMatcher(index_params, search_params)

while (cap.isOpened()):
  ret, frame = cap.read()
  if not ret:
    break
  
  img2 = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  kp_f, des_f = sift.detectAndCompute(img2, None)
  if des_f is None or des_t is None:
    cv2.imshow("img3", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
      break
    continue
  if des_f.dtype != np.float32:
    des_f = des_f.astype(np.float32)

  matches = flann.knnMatch(des_t, des_f, k=2)
  good = []
  for m,n in matches:
    if m.distance < 0.7*n.distance:
      good.append(m)
  if len(good) > MIN_MATCH_COUNT:
    src_pts = np.float32([ kp_t[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp_f[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    if mask is not None:
      matchesMask = mask.ravel().tolist()
    else:
      matchesMask = None
    h,w = img.shape
    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
    if M is not None:
      dst = cv2.perspectiveTransform(pts,M)
      frame = cv2.polylines(frame,[np.int32(dst)],True,255,3, cv2.LINE_AA)
  else:
    print("Not enough matches are found - %d/%d" % (len(good), MIN_MATCH_COUNT))
    matchesMask = None

  draw_params = dict(matchColor = (0,255,0), singlePointColor = None, matchesMask = matchesMask, flags = 2)
  img3 = cv2.drawMatches(img,kp_t,img2,kp_f,good,None,**draw_params)
  cv2.imshow("img3", img3)
  if cv2.waitKey(1) & 0xFF == ord('q'):
    break

cap.release()
cv2.destroyAllWindows()

Not enough matches are found - 7/10
Not enough matches are found - 8/10
Not enough matches are found - 10/10
Not enough matches are found - 7/10
Not enough matches are found - 10/10
Not enough matches are found - 6/10


: 